Wczytanie modułów


In [323]:
import lightgbm as lgb
import numpy as np
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
import json
from sklearn.metrics import precision_score, classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

Wczytanie danych


In [ ]:
data = pd.read_csv('data.csv')
#data = data_csv.sample(n=len(data_csv))
data['bmi'].fillna(data['bmi'].mean(),inplace=True)

data.head(25)

<ipython-input-289-099750243473>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['bmi'].fillna(data['bmi'].mean(),inplace=True)


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.600000,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.893237,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.500000,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.400000,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.000000,never smoked,1
5,56669,Male,81.0,0,0,Yes,Private,Urban,186.21,29.000000,formerly smoked,1
6,53882,Male,74.0,1,1,Yes,Private,Rural,70.09,27.400000,never smoked,1
7,10434,Female,69.0,0,0,No,Private,Urban,94.39,22.800000,never smoked,1
8,27419,Female,59.0,0,0,Yes,Private,Rural,76.15,28.893237,Unknown,1
9,60491,Female,78.0,0,0,Yes,Private,Urban,58.57,24.200000,Unknown,1


Przygotowanie danych

In [ ]:
col = ["gender", "ever_married", "work_type", "Residence_type", "smoking_status"]

data[col] = data[col].astype("category")


data.info()


numerical_features = ["age","hypertension","heart_disease", "avg_glucose_level", "bmi"]
categorical_features = ["gender", "ever_married", "work_type", "Residence_type", "smoking_status"]
target = "stroke"

X = data[numerical_features + categorical_features]
y = data[target]




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id                 5110 non-null   int64   
 1   gender             5110 non-null   category
 2   age                5110 non-null   float64 
 3   hypertension       5110 non-null   int64   
 4   heart_disease      5110 non-null   int64   
 5   ever_married       5110 non-null   category
 6   work_type          5110 non-null   category
 7   Residence_type     5110 non-null   category
 8   avg_glucose_level  5110 non-null   float64 
 9   bmi                5110 non-null   float64 
 10  smoking_status     5110 non-null   category
 11  stroke             5110 non-null   int64   
dtypes: category(5), float64(3), int64(4)
memory usage: 305.3 KB


Trenowanie modelu

In [ ]:
pd.options.mode.chained_assignment = None
encoder = sklearn.preprocessing.OrdinalEncoder()


X[categorical_features] = encoder.fit_transform(X[categorical_features])

print(X.head(25))

smote = SMOTE(random_state = 10)
X1, Y1 = smote.fit_resample(X, y)
Y1 = pd.DataFrame(Y1)
X1 = pd.DataFrame(X1)
print(Y1.value_counts)


X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.2, random_state=200)

classifier = lgb.LGBMClassifier()

# Trenowanie modelu na danych treningowych
gbm = classifier.fit(X_train, y_train)




     age  hypertension  heart_disease  avg_glucose_level        bmi  gender  \
0   67.0             0              1             228.69  36.600000     1.0   
1   61.0             0              0             202.21  28.893237     0.0   
2   80.0             0              1             105.92  32.500000     1.0   
3   49.0             0              0             171.23  34.400000     0.0   
4   79.0             1              0             174.12  24.000000     0.0   
5   81.0             0              0             186.21  29.000000     1.0   
6   74.0             1              1              70.09  27.400000     1.0   
7   69.0             0              0              94.39  22.800000     0.0   
8   59.0             0              0              76.15  28.893237     0.0   
9   78.0             0              0              58.57  24.200000     0.0   
10  81.0             1              0              80.43  29.700000     0.0   
11  61.0             0              1             12

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [ ]:
y_train.value_counts()

,count
stroke,
1,3905
0,3872


lightGBM


In [317]:
y_pred = classifier.predict(X_test)
precision_score(y_test, y_pred)
y_probs = classifier.predict_proba(X_test)
confidence = [max(probs) for probs in y_probs]


print(classification_report(y_test, y_pred))


    age  hypertension  heart_disease  avg_glucose_level  gender  ever_married  \
0  67.0             0              1             228.69     1.0           1.0   
1  61.0             0              0             202.21     0.0           1.0   
2  80.0             0              1             105.92     1.0           1.0   
3  49.0             0              0             171.23     0.0           1.0   
4  79.0             1              0             174.12     0.0           1.0   

   work_type  Residence_type    bmi  smoking_status  
0        2.0             1.0  240.0             1.0  
1        3.0             0.0  162.0             2.0  
2        2.0             0.0  199.0             2.0  
3        2.0             1.0  218.0             3.0  
4        3.0             0.0  113.0             2.0  
[0.9994517071691045, 0.8406307310797934, 0.997774235572555, 0.998231454059516, 0.9686580140999281, 0.9904553418416108, 0.9633130703075018, 0.9536020622599166, 0.9834332182432669, 0.99683675

In [ ]:
with open("model.pkl", "wb") as fout:
    pickle.dump(gbm, fout)

In [303]:
test_prob = pd.DataFrame({"age": 67, "hypertension": 0, "heart_disease": 1, "avg_glucose_level": 228.69, "gender": 1.0, "ever_married": 1.0, "work_type": 2.0, "Residence_type": 1.0, "bmi": 36.6, "smoking_status": 1.0}, index=[0])

classifier.predict(test_prob)
classifier.predict_proba(test_prob)

array([[0.97488444, 0.02511556]])

SVC

In [321]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
classification_report

clf = make_pipeline(StandardScaler(), SVC(gamma='auto', probability=True))
clf.fit(X_train, y_train)
svc = clf.predict(X_test)
clf.score(X, y, sample_weight=None)
print(classification_report(y_test, svc))
#clf.predict_proba(test_prob)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.90      0.89      0.90       989
           1       0.89      0.90      0.89       956

    accuracy                           0.90      1945
   macro avg       0.90      0.90      0.90      1945
weighted avg       0.90      0.90      0.90      1945



Random Forest Classifier


In [315]:
RF = RandomForestClassifier(n_estimators=1000, random_state=47 , n_jobs = 3)
RF.fit(X_train,y_train)
Y_Res_RF=RF.predict(X_test)
RF_Accuracy = accuracy_score(y_test, Y_Res_RF)

print(classification_report(y_test, Y_Res_RF))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


              precision    recall  f1-score   support

           0       0.96      0.99      0.97       989
           1       0.99      0.95      0.97       956

    accuracy                           0.97      1945
   macro avg       0.97      0.97      0.97      1945
weighted avg       0.97      0.97      0.97      1945



Decision Tree Classifier

In [320]:
DTC = DecisionTreeClassifier()
DTC.fit(X_train,y_train)
Y_pred_DTC = DTC.predict(X_test)
print(classification_report(y_test, Y_pred_DTC))


              precision    recall  f1-score   support

           0       0.94      0.93      0.93       989
           1       0.92      0.94      0.93       956

    accuracy                           0.93      1945
   macro avg       0.93      0.93      0.93      1945
weighted avg       0.93      0.93      0.93      1945



Logistic Regression

In [322]:
LR = LogisticRegression()
LR.fit(X_train,y_train)
Y_Res_LR=LR.predict(X_test)
print(classification_report(y_test, Y_Res_LR))


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.81      0.77      0.79       989
           1       0.77      0.82      0.80       956

    accuracy                           0.79      1945
   macro avg       0.79      0.79      0.79      1945
weighted avg       0.79      0.79      0.79      1945



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


KNN

In [325]:
KNN_clf = KNeighborsClassifier()
KNN_clf.fit(X_train,y_train)
Y_Res_KNN=KNN_clf.predict(X_test)
print(classification_report(y_test, Y_Res_KNN))


/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


              precision    recall  f1-score   support

           0       0.98      0.79      0.87       989
           1       0.82      0.98      0.89       956

    accuracy                           0.88      1945
   macro avg       0.90      0.89      0.88      1945
weighted avg       0.90      0.88      0.88      1945

